In [1]:
#censored. ;)
db_user = None
db_host = None
db_pass = None
db_name = None
data_dir = '/home/ckuethe/ipython/satellite_data/daily-tle-download/'

In [2]:
import mysql.connector

from zipfile import ZipFile
from bz2 import BZ2File
import json
from dateutil.parser import parse as dateparser
import re
import datetime
from math import pi, degrees

# sgp4
from sgp4.io import twoline2rv, verify_checksum, fix_checksum
from sgp4.earth_gravity import wgs84, wgs72
# OrbitalPy
from orbital.utilities import eccentric_anomaly_from_mean, ConvergenceError

import csv
import arrow
import os
from math import pi

# https://github.com/cbassa/twoline
from twoline import twoline

In [3]:
catdir = os.path.join(data_dir, 'satcat')
catfile = sorted(os.listdir(catdir))[-1]

satcat = dict()
with BZ2File(os.path.join(catdir, catfile)) as fd:
    tmp = json.load(fd)
    for rec in tmp:
        satcat[rec['INTLDES']] = satcat[str(rec['NORAD_CAT_ID'])] = satcat[int(rec['NORAD_CAT_ID'])] = rec

print(len(satcat))

136815


In [4]:
def spacetrack_record_build(line0, line1, line2,
                            query_date=None,
                            classification=None,
                            originator='UNKNOWN',
                            filename='',
                            skip_analyst=True,
                            satcat = {}):
    '''Given a TLE, construct an object that looks like what would be returned by the Space-Track API'''
    #Death to undocumented magic numbers
    F_OBJNUM = 1
    F_INTLDES = 2
    F_EPOCH = 3

    HOURS_PER_DAY = 24.0
    MINUTES_PER_DAY = 1440.0
    SECONDS_PER_DAY = 86400.0

    line1 = str(line1.strip())
    line2 = str(line2.strip())

    cat_num = int(line1[2:7])
    # "Analyst" satellites, possibly with characterized orbits, but no identification
    if skip_analyst and cat_num > 80000:
        return None

    if line0 is None:
        line0 = satcat.get(cat_num, {'OBJECT_NAME':''})['OBJECT_NAME']
    else:
        line0 = str(line0.strip())

    # occasionally I get TLEs with no checksum :(
    if len(line1) != 69 or len(line2) != 69:
        line1 = fix_checksum(line1)
        line2 = fix_checksum(line2)

    # because JSPOC doesn't aways left-justify the launch piece...
    intldes = line1[9:17].replace(' ', '')
    try:
        obj_id = intldes_to_objid(intldes)
    except AttributeError:
        return None
    # because python-sgp4 doesn't do this for me
    try:
        rev = int(line2[64:69])
    except ValueError:
        rev = 0

    if len(line0) == 0:
        # Unknown, make something up.
        line0 = "0 UNKNOWN-{}".format(intldes)

    if type(query_date) is not datetime.datetime:
        query_date = datetime.datetime.now()

    #ep_time = epoch_to_datetime(line1[18:32])

    if classification is None:
        classification = line1[7]
    classification = classification.upper()

    name = re.sub('^0 ', '', str(line0))
    try:
        satellite = twoline2rv(line1, line2, wgs72)
    except ValueError:
        return
    
    _n = satellite.no * MINUTES_PER_DAY / (2.0 * pi)

    if name.startswith('OBJECT '):
        obj_type = 'TBA'
    elif re.search(r' DEB(RIS)?$', name):
        obj_type = 'DEBRIS'
    elif re.search(r' r(/?b)?$', name, re.I):
        obj_type = 'ROCKET BODY'
    else:
        obj_type = 'PAYLOAD'

    try:
        ly = int(obj_id.split('-')[0])
    except ValueError:
        ly = None

    st_rec = {
        'NORAD': satellite.satnum,
        'BSTAR': satellite.bstar,
        #'CLASSIFICATION_TYPE': classification,
        # 'CLASSIFIED': classified,
        'ORIGINATOR': originator,
        'DATAFILE': filename,
        'OBJECT_TYPE': obj_type,
        'OBJECT_ID': obj_id,
        'NAME': name,
        'LAUNCH_YEAR': ly,
        #'INTLDES': intldes,
        #'TLE_LINE0': line0,
        'TLE1': line1,
        'TLE2': line2,
        'ARGP': degrees(satellite.argpo),
        'BSTAR': satellite.bstar,
        'RAAN': degrees(satellite.nodeo),
        'REV_AT_EPOCH': rev,
        'ECCENTRICITY': satellite.ecco,
        'INCLINATION': degrees(satellite.inclo),
        'MEAN_MOTION': _n,
        'MEAN_MOTION_DOT': satellite.ndot,
        'MEAN_MOTION_DDOT': satellite.nddot,
        'MEAN_ANOMALY': degrees(satellite.mo),
        'EPOCH': satellite.epoch, # - datetime.timedelta(microseconds=satellite.epoch.microsecond),
    }
    try:
        st_rec['TRUE_ANOMALY'] = float(eccentric_anomaly_from_mean(satellite.ecco, satellite.mo))
    except ConvergenceError:
        pass

    # from the SpaceTrack FAQ:
    '''
    We added semi-major axis, period, apogee, and perigee to the TLE and TLE_latest
    API classes so that users can filter their queries by these values, download only
    the data they need, and decrease the amount of the site's bandwidth that they use.
    Now, all the orbital elements in the satellite catalog (SATCAT) are available in the
    TLE class. However, the value of the same element (e.g. apogee) may not match exactly. 

    Every TLE already displays a value for the object's mean motion ("n") and eccentricity
    ("e"), so we derive these additional four values using the following calculations:

    period = 1440/n
    Using mu, the standard gravitational parameter for the earth (398600.4418),
    semi-major axis "a" = (mu/(n*2*pi/(24*3600))^2)^(1/3)
    Using semi-major axis "a", eccentricity "e", and the Earth's radius in km,
    apogee = (a * (1 + e))- 6378.135
    perigee = (a * (1 - e))- 6378.135
    '''

    _mu = 398600.4418
    _earth_rad = 6378.135
    _a = (_mu/(_n * 2.0 * pi /SECONDS_PER_DAY) ** 2 ) ** (1.0/3.0)
    st_rec['SEMIMAJOR'] = _a
    st_rec['PERIOD'] = MINUTES_PER_DAY / _n
    st_rec['APOGEE'] = (_a * (1 + satellite.ecco)) - _earth_rad
    st_rec['PERIGEE'] = (_a * (1 - satellite.ecco)) - _earth_rad

    return st_rec

def spacetrack_record_fix(row):
    for k in ['EPHEMERIS_TYPE', 'ELEMENT_SET_NO', 'COMMENT', 'ORDINAL']:
        row.pop(k, '')

    for k in row:
        row[str(k)] = str( row.pop(k, ''))

    for k in ['RA_OF_ASC_NODE', 'ARG_OF_PERICENTER', 'MEAN_MOTION_DOT', 'BSTAR',
              'INCLINATION', 'APOGEE', 'PERIGEE', 'MEAN_MOTION', 'MEAN_ANOMALY',
              'MEAN_MOTION_DDOT', 'PERIOD', 'ECCENTRICITY', 'SEMIMAJOR_AXIS']:
        try:
            row[k] = float(row[k])
        except (ValueError, KeyError):
            pass

    for k in ['OBJECT_NUMBER', 'NORAD_CAT_ID', 'REV_AT_EPOCH', 'EPOCH_MICROSECONDS']:
        try:
            row[k] = int(row[k])
        except (ValueError, KeyError):
            pass
    try:
        row['EPOCH'] = dateparser(row['EPOCH']) + datetime.timedelta(microseconds=row['EPOCH_MICROSECONDS'])
    except:
        pass
    
    return row

def intldes_to_objid(intldes):
    '''Given an International Designator YYLLLPPP build a prettier object id: CCYY-LLLPPP'''
    obj_id = list(re.match('^(\d{2})(\w+)$',intldes).groups())
    obj_id[0] = int(obj_id[0])
    if obj_id[0] < 56:
        obj_id[0] += 2000
    elif obj_id[0] <= 99:
        obj_id[0] += 1900
    obj_id = '{}-{}'.format(*obj_id)
    return obj_id

def load_zipped_tle_file(f, n=0):
    zip_file = ZipFile(f, 'r')
    nlist = zip_file.namelist()
    if len(nlist) > 1 and n == 0:
        warnings.warn("Zip archive contains {} members, processing only the first element".format(len(nlist)))

    inner_filename = nlist[n]
    zip_info = zip_file.getinfo(inner_filename)
    data = zip_file.read(inner_filename).decode()
    d = dateparser('{:04}/{:02}/{:02} {:02}:{:02}:{:02}'.format(*zip_info.date_time))
    tles = tle_buf_parser(data)
    return d, tles

def tle_buf_parser(buf):
    '''take a str representing a file of TLEs and turn it into 3 element tuples representing a satellite'''
    pat = r'''\S.+?[\r\n]{1,2} 1\s .*? [0-9] [\r\n]{1,2} 2\s .*? [0-9] [\r\n]{1,2}'''
    rgx = re.compile(pat,re.I|re.M|re.S|re.X)
    matches = re.findall(rgx, buf)
    matches = list(map(lambda x: x.strip(), matches))
    return matches


In [5]:
def ins_dict_rec(dbh, table, rec):
    '''using the cursor, insert a record into the database'''
    
    placeholders = ['%({})s'.format(i) for i in rec.keys()]
    sql = 'INSERT IGNORE INTO {} ( {} ) VALUES ( {} )'.format(table, ', '.join(list(rec.keys())), ', '.join(placeholders))
    curs = dbh.cursor()
    try:
        curs.execute(sql, rec)
        dbh.commit()
    except Exception:
        pass
    finally:
        curs.close()

def ins_dictlist_rec(dbh, table, reclist):
    '''using the database handle, insert a record into the database'''
    if reclist is None or reclist == []:
        return
    kz = list(reclist[0].keys())
    placeholders = ['%({})s'.format(i) for i in kz]
    sql = 'INSERT IGNORE INTO {} ( {} ) VALUES ( {} )'.format(table, ', '.join(kz), ', '.join(placeholders))
    curs = dbh.cursor()
    curs.execute("set rocksdb_max_row_locks=2000000;")

    try:
        curs.executemany(sql, reclist)
        dbh.commit()
    except Exception:
        pass
    finally:
        curs.close()

def getloaded(dbh):
    curs = dbh.cursor()
    curs.execute('select distinct(datafile) from tles;')
    loaded = set( [x[0] for x in curs.fetchall()])
    curs.close()
    assert(len(loaded))
    return loaded



In [6]:
# dbh = mariadb.connect(db_host, db_user, db_pass, db_name)
dbh = mysql.connector.connect(user=db_user, passwd=db_pass, host=db_host, port=3306, database=db_name, autocommit=False)

In [7]:
create_sql = '''
DROP TABLE IF EXISTS {0};
CREATE TABLE {0} (
    _rowid INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
    -- Identification
    object_id CHAR(11) NOT NULL, -- YYYY-LLLPPP
    norad INT NOT NULL,
    name VARCHAR(69), -- All the other tle lines can be 69 characters, why not this too?
    originator VARCHAR(64), -- where did this TLE come from?
    launch_year YEAR NOT NULL,
    object_type ENUM('PAYLOAD', 'BOOSTER', 'DEBRIS', 'FUEL', 'UNKNOWN'),
    epoch DATETIME(6) NOT NULL, -- TIMESTAMP is unixtime which can't grok really old dates
    datafile VARCHAR(64) DEFAULT NULL,

        -- Keps
        eccentricity DOUBLE NOT NULL,
        inclination DOUBLE NOT NULL,
        semimajor DOUBLE NOT NULL,
        raan DOUBLE NOT NULL,
        argp DOUBLE NOT NULL,
        mean_anomaly DOUBLE NOT NULL,
        true_anomaly DOUBLE NOT NULL,
        bstar DOUBLE NOT NULL,
        mean_motion DOUBLE NOT NULL,
        mean_motion_dot DOUBLE NOT NULL,
        mean_motion_ddot DOUBLE NOT NULL,

        -- Derived Parameters
        rev_at_epoch INT NOT NULL,
        apogee FLOAT NOT NULL,
        perigee FLOAT NOT NULL,
        period FLOAT NOT NULL,

        -- TLE
        tle1 CHAR(70) NOT NULL,
        tle2 CHAR(70) NOT NULL

    );

    CREATE INDEX idx_year on {0}(launch_year);
    CREATE INDEX idx_datafile on {0}(datafile);
    CREATE INDEX idx_epoch on {0}(epoch);
    CREATE INDEX idx_objid on {0}(object_id);
    CREATE INDEX idx_norad on {0}(norad);
    CREATE INDEX idx_name on {0}(name);
    CREATE UNIQUE INDEX idx_dedup on {0}(norad, epoch);

'''

#if True:
curs = dbh.cursor()
resp = curs.execute(create_sql.format('tles'), multi=True)

# curs.close()
# dbh.commit()

In [10]:
lr = list(resp)

In [11]:
lr

[]

In [8]:
print(create_sql.format('vimpel'))


DROP TABLE IF EXISTS vimpel;
CREATE TABLE vimpel (
    _rowid INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
    -- Identification
    object_id CHAR(11) NOT NULL, -- YYYY-LLLPPP
    norad INT NOT NULL,
    name VARCHAR(69), -- All the other tle lines can be 69 characters, why not this too?
    originator VARCHAR(64), -- where did this TLE come from?
    launch_year YEAR NOT NULL,
    object_type ENUM('PAYLOAD', 'BOOSTER', 'DEBRIS', 'FUEL', 'UNKNOWN'),
    epoch DATETIME(6) NOT NULL, -- TIMESTAMP is unixtime which can't grok really old dates
    datafile VARCHAR(64) DEFAULT NULL,

        -- Keps
        eccentricity DOUBLE NOT NULL,
        inclination DOUBLE NOT NULL,
        semimajor DOUBLE NOT NULL,
        raan DOUBLE NOT NULL,
        argp DOUBLE NOT NULL,
        mean_anomaly DOUBLE NOT NULL,
        true_anomaly DOUBLE NOT NULL,
        bstar DOUBLE NOT NULL,
        mean_motion DOUBLE NOT NULL,
        mean_motion_dot DOUBLE NOT NULL,
        mean_motion_ddot DOUBLE NOT NULL,

   

In [9]:
dfz = !ls -r satellite_data/daily-tle-download/classified/classified_2020*zip
loaded = getloaded(dbh)

for datafile in dfz[:15]:
    shortname = datafile.replace('satellite_data/','').replace('daily-tle-download/','')
    if shortname in loaded:
        continue
    print("load", shortname)
    ftime, tles = load_zipped_tle_file(datafile)
    l = [spacetrack_record_build(*t.split('\r\n'), filename=shortname, skip_analyst=False, originator='seesat') for t in tles]
    l = list(filter(lambda x: x is not None, l))

    ins_dictlist_rec(dbh, 'tles', l)

load classified/classified_20200505071702.zip
load classified/classified_20200425071701.zip


In [10]:
dfz = !ls satellite_data/daily-tle-download/supplemental/*2020*txt

orgs = {
    'gps': 'navcen',
    'glonass': 'roscosmos',
    'meteosat': 'eumetsat',
    'iss': 'nasa',
    'ses': 'ses',
    'orbcomm': 'orbcomm',
    'starlink': 'spacex',
    'trusat_all': 'trusat',
    'oneweb': 'oneweb',
    'intelsat': 'intelsat',
}

loaded = getloaded(dbh)


for datafile in dfz:
    shortname = datafile.replace('satellite_data/','').replace('daily-tle-download/','')
    if shortname in loaded:
        continue
    print("load", shortname)

    o = shortname.split('/')[-1]
    o = o.split('-')[0]
    o = orgs[o]
    
    with open(datafile) as fd:
        tles = tle_buf_parser(fd.read())
        l = [spacetrack_record_build(*re.split('[\r\n]+', t, re.S|re.M), filename=shortname, skip_analyst=True, originator=o) for t in tles]
        l = list(filter(lambda x: x is not None, l))
        ins_dictlist_rec(dbh, 'tles', l)
    dbh.commit()

load supplemental/glonass-20200110233908.txt
load supplemental/glonass-20200113233838.txt
load supplemental/glonass-20200115234258.txt
load supplemental/glonass-20200120233856.txt
load supplemental/glonass-20200122234022.txt
load supplemental/glonass-20200125234150.txt
load supplemental/glonass-20200206233856.txt
load supplemental/glonass-20200209233854.txt
load supplemental/glonass-20200216234252.txt
load supplemental/glonass-20200218234348.txt
load supplemental/glonass-20200223234248.txt
load supplemental/glonass-20200225234320.txt
load supplemental/glonass-20200302234346.txt
load supplemental/glonass-20200307233944.txt
load supplemental/glonass-20200311004420.txt
load supplemental/glonass-20200314004442.txt
load supplemental/glonass-20200320004058.txt
load supplemental/glonass-20200402004010.txt
load supplemental/glonass-20200403004006.txt
load supplemental/glonass-20200408004528.txt
load supplemental/glonass-20200414004228.txt
load supplemental/glonass-20200422004638.txt
load suppl

In [11]:
dfz = !ls -r satellite_data/daily-tle-download/spacetrack/spacetrack_2020*bz2
loaded = getloaded(dbh)

for datafile in dfz[:15]:
    shortname = datafile.replace('satellite_data/','').replace('daily-tle-download/','')
    if shortname in loaded:
        continue
    print(shortname)
    with BZ2File(datafile) as bz:
        tles = json.load(bz)
        l = [spacetrack_record_build(t['TLE_LINE0'], t['TLE_LINE1'], t['TLE_LINE2'], filename=shortname, skip_analyst=True, originator='spacetrack') for t in tles]
        l = list(filter(lambda x: x is not None, l))
        ins_dictlist_rec(dbh, 'tles', l)
    dbh.commit()

In [12]:
dfz = !ls -r satellite_data/daily-tle-download/public/public_2020*zip

loaded = getloaded(dbh)

for datafile in dfz[:15]:
    shortname = datafile.replace('satellite_data/','').replace('daily-tle-download/','')
    if shortname in loaded:
        continue
    ftime, tles = load_zipped_tle_file(datafile)
    print(shortname)

    l = [spacetrack_record_build(*t.split('\r\n'), filename=shortname, skip_analyst=True, originator='18spcs') for t in tles]
    l = list(filter(lambda x: x is not None, l))

    ins_dictlist_rec(dbh, 'tles', l)
    dbh.commit()

In [13]:
print(str(datetime.datetime.now()))
1/0

2020-05-09 00:04:31.083852


ZeroDivisionError: division by zero

In [15]:
if True:
    curs = dbh.cursor()
    curs.execute('optimize table tles;')
    curs.fetchone()
    curs.close()
#    dbh.close()

In [13]:
dfz = !ls -r satellite_data/historical-tle/tle*bz2
loaded = getloaded(dbh)

for datafile in dfz:
    shortname = datafile.replace('satellite_data/','').replace('historical-tle/','')
    if shortname in loaded:
        continue
    print(shortname)
    with BZ2File(datafile) as bz:
        try:
            tles = json.load(bz)
        except json.JSONDecodeError:
            continue
        l = [spacetrack_record_build(t.get('TLE_LINE0', None), t['TLE_LINE1'], t['TLE_LINE2'], filename=shortname, skip_analyst=True, originator='spacetrack', satcat=satcat) for t in tles]
        l = list(filter(lambda x: x is not None, l))
        try:
            ins_dictlist_rec(dbh, 'tles', l)
        except mysql.connector.DatabaseError:
            pass
    dbh.commit()

tle_2010-01-19.json.bz2
tle_2010-01-18.json.bz2
tle_2010-01-17.json.bz2
tle_2010-01-16.json.bz2
tle_2010-01-15.json.bz2
tle_2010-01-14.json.bz2
tle_2010-01-13.json.bz2
tle_2010-01-12.json.bz2
tle_2010-01-11.json.bz2
tle_2010-01-10.json.bz2
tle_2010-01-09.json.bz2
tle_2010-01-08.json.bz2
tle_2010-01-07.json.bz2
tle_2010-01-06.json.bz2
tle_2010-01-05.json.bz2
tle_2010-01-04.json.bz2
tle_2010-01-03.json.bz2
tle_2010-01-02.json.bz2
tle_2010-01-01.json.bz2
tle_2009-12-31.json.bz2
tle_2009-12-30.json.bz2
tle_2009-12-29.json.bz2
tle_2009-12-28.json.bz2
tle_2009-12-27.json.bz2
tle_2009-12-26.json.bz2
tle_2009-12-25.json.bz2
tle_2009-12-24.json.bz2
tle_2009-12-23.json.bz2
tle_2009-12-22.json.bz2
tle_2009-12-21.json.bz2
tle_2009-12-20.json.bz2
tle_2009-12-19.json.bz2
tle_2009-12-18.json.bz2
tle_2009-12-17.json.bz2
tle_2009-12-16.json.bz2
tle_2009-12-15.json.bz2
tle_2009-12-14.json.bz2
tle_2009-12-13.json.bz2
tle_2009-12-12.json.bz2
tle_2009-12-11.json.bz2
tle_2009-12-10.json.bz2
tle_2009-12-09.j

In [14]:
dfz = !ls satellite_data/historicTLE_COPY/*tle
loaded = getloaded(dbh)

for datafile in dfz:
    shortname = datafile.replace('satellite_data/','').replace('historicTLE_COPY/','')
    if shortname in loaded:
        continue
    objnum = int(shortname.replace('.tle', ''))
    objname = satcat[objnum]['SATNAME'].strip()
    if objnum % 25 == 0:
        print(shortname, objname)
    with open(datafile) as fd:
        buf = fd.readlines()
        l = []
        try:
            while True:
                l.append( spacetrack_record_build(objname, buf[0], buf[1], filename=shortname, originator='spacetrack'))
                buf.pop(1)
                buf.pop(0)
        except IndexError:
            pass
        try:
            ins_dictlist_rec(dbh, 'tles', l)
        except mysql.connector.DatabaseError:
            pass
    dbh.commit()

10000.tle OPS 9437 (DSCS 2-7)
1000.tle TITAN 3A DEB (TRUSS)
10025.tle ATLAS CENTAUR R/B
10050.tle COSMOS 838 DEB
10075.tle DELTA 1 DEB
100.tle DISCOVERER 23
10100.tle COSMOS 838 DEB
10125.tle COSMOS 838 DEB
10150.tle COSMOS 931
10175.tle COSMOS 934 DEB
10200.tle DELTA 1 DEB
10225.tle DELTA 1 DEB
10250.tle DELTA 1 DEB
10275.tle COSMOS 936 DEB
10300.tle SL-4 R/B
10325.tle COSMOS 938 DEB
10350.tle COSMOS 913 DEB
10375.tle COSMOS 956
10400.tle COSMOS 952 DEB (ANTENNA)
10450.tle COSMOS 839 DEB
10475.tle DELTA 1 DEB
10500.tle SL-4 DEB
10525.tle SL-8 DEB
10550.tle COSMOS 969 DEB
10575.tle COSMOS 970 DEB
10600.tle SL-8 R/B
10625.tle DELTA 1 DEB
10650.tle DELTA 1 DEB
10675.tle N-1 R/B
10700.tle SL-4 R/B
10725.tle COSMOS 993
10750.tle DELTA 1 DEB
10775.tle SL-13 DEB
10825.tle DELTA 1 DEB
10850.tle SL-4 DEB
10875.tle DELTA 1 DEB
10900.tle SL-8 DEB
10925.tle MOLNIYA 1-40
10950.tle DELTA 1 R/B(2)
10975.tle COMSTAR 3
11000.tle SL-4 R/B
1100.tle COSMOS 57 DEB
11050.tle COSMOS 1041
11075.tle SL-6 R/B(

In [ ]:
def orbital_properties(semimajor_axis, e):
    earth_radius = 6378.135
    mu = 398600.4418
    seconds_per_day = 86400.0
    minutes_per_day = seconds_per_day / 60.0

    apogee = (semimajor_axis * (1 + e)) - earth_radius
    perigee = (semimajor_axis * (1 - e)) - earth_radius
    period = 2.0 * pi * (semimajor_axis**3 / mu) ** 0.5
    mean_motion = seconds_per_day / period

    return (apogee, perigee, period/60.0, mean_motion)

In [ ]:
df = 'vimpel/orbits/orbits.20200420.txt'
fd = open(df)

fz = ['line_num','objnum','firstseen', 'epoch','obs_age','semimajor','inclination','raan','eccentricity','arglat','argp','drag_area','magnitude','time_uncert','pos_uncert']

rd = csv.DictReader(fd,fz)

In [ ]:
q = 'select * from tles where norad <> 25544 order by epoch desc limit 1;'
curs = dbh.cursor(dictionary=True)
curs.execute(q)
l = curs.fetchone()
print(l)
curs.close()

In [ ]:
def vimpel2spacetrack(vrec, df):
    r = {'originator': 'vimpel', 'datafile': df, 'object_type': 'UNKNOWN', 'name': None}
    for f in ['objnum', 'obs_age']:
        r[f] = int(vrec[f])
    for f in ['semimajor', 'inclination', 'raan', 'arglat', 'argp', 'eccentricity', 'magnitude', 'drag_area', 'time_uncert', 'pos_uncert']:
        try:
            r[f] = float(vrec[f])
        except ValueError:
            r[f] = None
    
    r['epoch'] = arrow.get(vrec['epoch'], 'DDMMYYYY HHmmss').datetime
    r['firstseen'] = arrow.get(vrec['firstseen'], 'DDMMYYYY').datetime.date()
    r['anomaly'] = r['arglat'] - r['argp']
    if r['anomaly'] < 0.0:
        r['anomaly'] += 360.0
    
    r['apogee'], r['perigee'], r['period'], r['mean_motion'] = orbital_properties(r['semimajor'], r['eccentricity'])

    ey, ed = twoline.datetime_to_epoch(r['epoch'])
    r['name'], r['tle1'], r['tle2'] = twoline.format_tle(r['objnum'], ey, ed, incl=r['inclination'], node=r['raan'], ecc=r['eccentricity'], argp=r['argp'], m=0.0, n=r['mean_motion'], name='vimpel {}'.format(r['objnum']), desig='{}A'.format(r['objnum']))
    
    for f in ['obs_age', 'arglat']:
        r.pop(f, None)
    return r

In [ ]:
l = []
for r in rd:
    r = dict(r)
    l.append(vimpel2spacetrack(r, df))


In [ ]:
l[0]

In [ ]:
ins_dictlist_rec(dbh, 'vimpel', l)